Transform video to audio

In [2]:
from audio_extractor import extract_audio

debra_video = "C:/Users/iggym/Documents/Movies/Baby Driver (2017) [YTS.AG]/Soundtrack/[ONTIVA.COM] Debra-144p.mp4"
debra_audio_file = "debra_song"

baby_driver_video_file = "C:/Users/iggym/Documents/Movies/Baby Driver (2017) [YTS.AG]/Baby.Driver.2017.720p.BluRay.x264-[YTS.AG].mp4"
baby_driver_audio_file = "baby_driver_audio"

sample_rate = 22000

extract_audio(debra_video, sample_rate, debra_audio_file)
extract_audio(baby_driver_video_file, sample_rate, baby_driver_audio_file)

C:/Users/iggym/Documents/Recuperacion de Informacion Multimedia/rim_project/debra_song.22000.wav
Audio extracted: C:/Users/iggym/Documents/Recuperacion de Informacion Multimedia/rim_project/debra_song.22000.wav
debra_song.22000.wav
baby_driver_audio.22000.wav


'baby_driver_audio.22000.wav'

Generate Descriptors

In [3]:
from audio_descriptor_generator import create_audio_descriptors

dimension = 32
window = 2200
hop = 2200

baby_driver_audio_file = "baby_driver_audio.22000.wav"
debra_song_audio_file = "debra_song.22000.wav"

debra_descriptors = create_audio_descriptors(debra_song_audio_file, sample_rate, dimension, window, hop)
print(debra_descriptors.shape)
baby_descriptors = create_audio_descriptors(baby_driver_audio_file, sample_rate, dimension, window, hop)
print(baby_descriptors.shape)

baby_driver_descriptors_file = "baby_driver_audio_descriptors.bin"
debra_song_descriptors_file = "debra_song_descriptors.bin"

debra_descriptors.tofile(debra_song_descriptors_file, sep="\n")
baby_descriptors.tofile(baby_driver_descriptors_file, sep="\n")

c:\Users\iggym\Documents\Recuperacion de Informacion Multimedia\rim_project\venv\lib\site-packages\librosa\core\audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


NoBackendError: 

Calculate distances

In [5]:
import numpy
import time
from scipy.spatial import distance

baby_driver_descriptors_file = "baby_driver_audio_descriptors.bin"
# debra_song_descriptors_file = "debra_song_descriptors.bin"
debra_song_descriptors_file = "new debra song descriptors.bin"

debra_song_shape = (3434, 32)

song_descriptors = numpy.fromfile(debra_song_descriptors_file, sep="\n").reshape(debra_song_shape)
print(song_descriptors.shape)

movie_shape = (67598, 32)
movie_descriptors = numpy.fromfile(baby_driver_descriptors_file, sep="\n").reshape(movie_shape)
print(movie_descriptors.shape)

t0 = time.time()
distances = distance.cdist(song_descriptors, movie_descriptors)
t1 = time.time()

print(f"Distances {distances.shape} {round(t1-t0, 2)} secs")

(3434, 32)
(67598, 32)
Distances (3434, 67598) 12.7 secs


Get neighbours

In [ ]:
number_of_neighbours = 5

neighbours = []
t10 = time.time()
for song_descriptor in distances:
    neighs = numpy.argpartition(song_descriptors, number_of_neighbours)[:number_of_neighbours]
    neighbours.append(neighs)
t11 = time.time()
neighbours = numpy.array(neighbours)
print(f"Neighbours {neighbours.shape} {round(t11-t10, 2)} secs")